In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [2]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [4]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```


In [5]:
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [7]:
model = OpenAI(temperature=0)

In [10]:
_input = prompt.format_prompt(question="what's the capital of france")
print(_input)

text='answer the users question as best as possible.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```\nwhat\'s the capital of france'


In [11]:
output = model(_input.to_string())

In [12]:
output

'?\n\n```json\n{\n\t"answer": "Paris",\n\t"source": "https://www.worldatlas.com/articles/what-is-the-capital-of-france.html"\n}\n```'

In [13]:
output_parser.parse(output)

{'answer': 'Paris',
 'source': 'https://www.worldatlas.com/articles/what-is-the-capital-of-france.html'}

In [14]:
chat_model = ChatOpenAI(temperature=0)

In [15]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [16]:
_input = prompt.format_prompt(question="what's the capital of france")
print(_input)


messages=[HumanMessage(content='answer the users question as best as possible.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```\nwhat\'s the capital of france', additional_kwargs={}, example=False)]


In [18]:
output = chat_model(_input.to_messages())

In [19]:
output

AIMessage(content='```json\n{\n\t"answer": "Paris",\n\t"source": "https://en.wikipedia.org/wiki/Paris"\n}\n```', additional_kwargs={}, example=False)